<a href="https://colab.research.google.com/github/yogaaditandanu/BelajarDeepLearning/blob/main/multivariate_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                 infer_datetime_format=True, index_col='datetime', header=0)
df

/tmp/ipython-input-405942205.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


infer_datetime_format=True: Mencoba menebak format tanggal dan waktu secara otomatis (meskipun ada peringatan bahwa ini sudah usang).

index_col='datetime': Menjadikan kolom 'datetime' sebagai indeks (penanda baris) dari DataFrame. Ini sangat penting untuk data time series.

header=0: Menunjukkan bahwa baris pertama (indeks 0) dari file adalah header (nama kolom).

In [3]:
def normalize_series(data, min, max):
  data = data - min
  data = data / max
  return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [4]:
N_FEATURES = len(df.columns)

In [5]:
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

Data dibagi menjadi dua set: data latih (x_train) dan data validasi (x_valid).

SPLIT_TIME = int(len(data) * 0.5): Menentukan titik pembagian, yaitu 50% dari total data.

x_train = data[:SPLIT_TIME]: 50% data pertama digunakan untuk melatih model.

x_valid = data[SPLIT_TIME:]: 50% data sisanya digunakan untuk memvalidasi (menguji) model.

Penting: Untuk data time series, data harus dibagi secara kronologis (berurutan), tidak boleh diacak. Kita melatih model pada data masa lalu (x_train) dan mengujinya pada data masa depan (x_valid).

In [6]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
  ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
  return ds.batch(batch_size).prefetch(1)

Tujuannya adalah mengubah data serial (satu urutan panjang) menjadi format supervised learning (pasangan input-output).

n_past=24: Menentukan bahwa model akan melihat 24 data poin sebelumnya (input).

n_future=24: Menentukan bahwa model akan mencoba memprediksi 24 data poin ke depan (output/label).

ds = ds.map(lambda w: (w[:n_past], w[n_past:])): Baris ini adalah intinya. Ini mengubah setiap "jendela" (window) data menjadi pasangan: (24 data poin pertama sebagai input, 24 data poin berikutnya sebagai target/label).

ds.batch(batch_size): Mengelompokkan data ke dalam batch (tumpukan) untuk efisiensi pelatihan.

In [7]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Ini mendefinisikan arsitektur model neural network menggunakan Keras (bagian dari TensorFlow).

tf.keras.models.Sequential: Model ini adalah tumpukan lapisan (layers) yang berurutan.

tf.keras.layers.Dense(64, ...): Layer pertama adalah Dense layer (terhubung penuh) dengan 64 neuron.

input_shape=(N_PAST, N_FEATURES): Ini memberi tahu model bahwa inputnya akan berbentuk (24, 7), yaitu 24 langkah waktu (N_PAST) dengan 7 fitur (N_FEATURES) di setiap langkah.

tf.keras.layers.Dense(32, activation='relu'): Layer tersembunyi (hidden layer) kedua dengan 32 neuron dan fungsi aktivasi 'relu'.

tf.keras.layers.Dense(N_FEATURES): Layer output dengan 7 neuron (N_FEATURES), karena model harus memprediksi 7 nilai fitur.

Model ini akan mengambil 24 langkah waktu sebagai input dan menghasilkan 24 langkah waktu sebagai output (prediksi).

In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True

callbacks = myCallback()

Kode ini membuat sebuah "aturan" khusus yang disebut callback.

on_epoch_end(...): Fungsi ini akan dijalankan di setiap akhir epoch (satu siklus pelatihan).

if (logs.get('mae') < 0.055 and ...): Aturan ini mengecek: JIKA error (MAE) pada data latih dan error (val_MAE) pada data validasi keduanya sudah di bawah 0.055,

self.model.stop_training = True: ...maka hentikan pelatihan.

Ini disebut Early Stopping (pemberhentian dini), yang berguna untuk mencegah overfitting dan menghemat waktu jika model sudah mencapai target akurasi.

In [10]:
# Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

model.compile(...): Mengkonfigurasi model sebelum pelatihan.

loss='mae': Menentukan loss function (fungsi kerugian) sebagai 'mae' (Mean Absolute Error). Ini adalah cara model mengukur seberapa salah prediksinya.

optimizer = tf.keras.optimizers.Adam(...): Memilih optimizer 'Adam'. Optimizer adalah algoritma yang digunakan untuk memperbarui model agar loss-nya semakin kecil (model semakin pintar).

metrics=["mae"]: Meminta model untuk melaporkan metrik 'mae' selama pelatihan.

In [11]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/100
   1346/Unknown 13s 9ms/step - loss: 0.0796 - mae: 0.0796

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - loss: 0.0796 - mae: 0.0796 - val_loss: 0.0596 - val_mae: 0.0596
Epoch 2/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - loss: 0.0607 - mae: 0.0607 - val_loss: 0.0570 - val_mae: 0.0570
Epoch 3/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 0.0591 - mae: 0.0591 - val_loss: 0.0576 - val_mae: 0.0576
Epoch 4/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - loss: 0.0592 - mae: 0.0592 - val_loss: 0.0572 - val_mae: 0.0572
Epoch 5/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 0.0582 - mae: 0.0582 - val_loss: 0.0580 - val_mae: 0.0580
Epoch 6/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - loss: 0.0585 - mae: 0.0585 - val_loss: 0.0582 - val_mae: 0.0582
Epoch 7/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0572 - val_mae: 0.0572
Epoch 8/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 0.0578 - mae: 0.0578 - val_loss: 0.0570 - val_mae: 0.0570
Epoch 9/100
1349/1349 ━━━━━━

model.fit(...): Ini adalah perintah untuk memulai proses pelatihan.

train_set: Data yang digunakan untuk melatih.

validation_data=(valid_set): Data yang digunakan untuk menguji performa model di setiap akhir epoch.

epochs=100: Model akan dilatih maksimal 100 kali (100 epoch).

callbacks=callbacks: Menggunakan aturan early stopping yang didefinisikan di Sel 9.

Output: Kita bisa lihat proses pelatihan. Pelatihan berhenti di Epoch 49 karena kondisi callback (val_mae < 0.055) terpenuhi.

In [13]:
train_pred = model.predict(train_set)
train_pred[0][0]
# Output
# array([ 3.8611603e-01,  2.5834844e-01,  2.5597256e-02,  3.8636056e-01,
#        -4.1037248e-04,  1.7979867e-03,  8.6604381e-01], dtype=float32)

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


array([ 0.38435915,  0.23825192,  0.0325161 ,  0.36953878, -0.00585023,
       -0.00310054,  0.8439041 ], dtype=float32)